In [0]:
#pip install faker

In [0]:
from faker import Faker
import pandas as pd
from datetime import datetime
from pandas import DataFrame
import numpy as np
from pyspark.sql.functions import *

In [0]:
def geracao_dados_fake(qtd_rows: int = 100, locale: str = "pt_BR") -> DataFrame:

    data_atual = datetime.now()
    local_faker = Faker(locale)
    list_result = []
    for _ in range(qtd_rows):
        data_nascimento = local_faker.date()
        dict_result = {
            "Nome": local_faker.name(),
            "NomOcupacao": local_faker.job(),
            "NomCidade": local_faker.country(),
            "NumTelefone": local_faker.phone_number(),
            "NomEmail": local_faker.ascii_email(),
            "DataNascimento": str(data_nascimento),
            'NumIdade': (data_atual.date() - datetime.strptime(data_nascimento, "%Y-%m-%d").date()).days //365
        }
        list_result.append(dict_result)

    return pd.DataFrame(list_result)

In [0]:
dfDadosFake = geracao_dados_fake(10000)

In [0]:
dfDadosFake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome            10000 non-null  object
 1   NomOcupacao     10000 non-null  object
 2   NomCidade       10000 non-null  object
 3   NumTelefone     10000 non-null  object
 4   NomEmail        10000 non-null  object
 5   DataNascimento  10000 non-null  object
 6   NumIdade        10000 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [0]:
num_linhas = 200

indices_aleatorios = np.random.choice(dfDadosFake.index, size=num_linhas, replace=False)

for i in indices_aleatorios:

    colunas_para_alterar = np.random.choice(['NomOcupacao','NomCidade'], size=2)

    for coluna in colunas_para_alterar:

        dfDadosFake[coluna][i] = np.nan

<command-3348275025660789>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDadosFake[coluna][i] = np.nan


In [0]:
dfDadosFake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome            10000 non-null  object
 1   NomOcupacao     9862 non-null   object
 2   NomCidade       9840 non-null   object
 3   NumTelefone     10000 non-null  object
 4   NomEmail        10000 non-null  object
 5   DataNascimento  10000 non-null  object
 6   NumIdade        10000 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [0]:
dfspark  = spark.createDataFrame(dfDadosFake)

In [0]:
type(dfspark)

Out[36]: pyspark.sql.dataframe.DataFrame

In [0]:
null_counts = dfspark.select([sum(col(c).isNull().cast("int")).alias(c) for c in dfspark.columns])

In [0]:
display(null_counts)

Nome,NomOcupacao,NomCidade,NumTelefone,NomEmail,DataNascimento,NumIdade
0,138,160,0,0,0,0


In [0]:
dfspark.dropna(subset=['NomOcupacao']).show(truncate=False)

+-----------------------+--------------------------+--------------------------------+-------------------+-------------------------------+--------------+--------+
|Nome                   |NomOcupacao               |NomCidade                       |NumTelefone        |NomEmail                       |DataNascimento|NumIdade|
+-----------------------+--------------------------+--------------------------------+-------------------+-------------------------------+--------------+--------+
|Srta. Ana Vitória Ramos|Confeiteiro               |Mauritânia                      |0800-911-5967      |diaspedro@cardoso.br           |2012-03-25    |11      |
|Letícia Alves          |Matemático                |Austrália                       |51 8642 1612       |joao-miguel63@nogueira.com     |2008-06-23    |15      |
|Srta. Isadora da Costa |Procurador de justiça     |Singapura                       |(031) 8872 0458    |duartepedro@freitas.br         |2013-09-08    |10      |
|Thiago Fernandes       |Cum

In [0]:
dfspark.dropna().count()

Out[40]: 9800

In [0]:
dfspark.na.drop(subset=['NomOcupacao']).count()

Out[41]: 9862

In [0]:
dfspark.na.drop().count()

Out[42]: 9800